# Intro

## Get API key

In [ ]:
import os
import getpass

# Set the environment variable
os.environ["COHERE_API_KEY"] = getpass.getpass("Enter your Cohere API key: ")

In [15]:
cohere_api_key = os.environ["COHERE_API_KEY"]
print(cohere_api_key)

nt9wg84wNjGfVEwC9ysQuAVdEJlT9bDYjtMeMA8g


In [30]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter openai api key: ")

## load Cohere model 

In [ ]:
from langchain_cohere.llms import Cohere

# model = Cohere(temperature=0.1)
model = Cohere(cohere_api_key=cohere_api_key, model="command", temperature=0.1)

In [69]:
from langchain_cohere import ChatCohere

chat = ChatCohere(cohere_api_key=cohere_api_key, model="command-r-plus", temperature=0.1)

## load openai model

In [31]:
from langchain_openai import OpenAI 

model_aopenai = OpenAI()

## invoke

In [70]:

message = "سلام به اسپانیایی چی میشه؟"

response = chat.invoke(message)

print(response.content)

¡Hola!


In [29]:
message = "ترجمه‌ی کلمه‌ی «دانشگاه» به آلمانی چی میشه؟"
# response = model.invoke(message)
print(response.content)

ترجمه کلمه "دانشگاه" به آلمانی "Universität" است.


In [ ]:
message = "ترجمه‌ی کلمه‌ی «دانشگاه» به آلمانی چی میشه؟"
response = model_aopenai.invoke(message)
print(response.content)

## Human & System message

In [71]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(
        content = "تو یک معلم زبان اسپانیایی هستی. پاسخ‌های دانش‌آموز را به صورت خلاصه و با یک مثال کوتاه و عامیانه بده "
    ),
    HumanMessage(
        content="سلام به اسپانیایی چی میشه؟"
    )
]

response = chat.invoke(messages)

In [72]:
print(response.additional_kwargs['token_count'])

{'input_tokens': 199.0, 'output_tokens': 65.0}


## Batch Processing

In [80]:
from langchain_core.messages import HumanMessage, SystemMessage

system = SystemMessage(
    content="تو یک معلم زبان اسپانیایی هستی. پاسخ‌های دانش‌آموز را به صورت خلاصه و با یک مثال کوتاه و عامیانه بده "
)

messages = [
    [system, HumanMessage(
        content="سلام به اسپانیایی چی میشه؟"
    )],
    [system, HumanMessage(
        content="چطور به اسپانیایی بگم من در شیراز متولد شدم؟"
    )]
] 

responses = chat.batch(messages)

In [78]:
for response in responses:
    print(response.content)

سلام در اسپانیایی "hola" (هولا) گفته می‌شود. مثلاً وقتی وارد یک اتاق می‌شوید و می‌خواهید به همه سلام کنید، می‌گویید: "¡Hola a todos!" (هولا آ تودوس!) که معنی آن "سلام به همه!" است.
شما می‌توانید بگویید:

"Nací en Teherán."

مثال:

من در تهران متولد شدم. = "Nací en Teherán."


## Better view - markdown

In [47]:
import markdown
from IPython.display import HTML, display

def print_md(input):
    input = markdown.markdown(input)
    content = "<div div=rtl>{}</div>".format(input)

    display(HTML(content))

In [ ]:
for response in responses:
    print_md(response.content)
    print('*'*70)

In [75]:
import markdown
from IPython.display import display, HTML

def print_md_v2(content):
    content = markdown.markdown(content)
    styled_content = f"""
    <div dir="rtl" style="font-family: Tahoma, Arial, sans-serif; font-size: 16px; color: #300; background-color: #f9f9f9; padding: 10px; border-radius: 8px;">
        {content}
    </div>
    """
    display(HTML(styled_content))


In [81]:
for response in responses:
    print_md_v2(response.content)
    print('*'*70)

**********************************************************************


**********************************************************************


In [55]:
print_md_v2("# سلام دنیا\n**این یک متن بولد است**")

In [63]:
import markdown
from IPython.display import display, HTML
import re

def fix_ltr(content):
    """Wraps LTR words in <bdi> tags to handle mixed-direction text properly."""
    return re.sub(r'([a-zA-Z0-9@#\-_]+)', r'<bdi>\1</bdi>', content)

def print_md_v3(content):
    content = markdown.markdown(content)
    content = fix_ltr(content)
    styled_content = f"""
    <div dir="rtl" style="font-family: Tahoma, Arial, sans-serif; font-size: 16px; color: #222; background-color: #fafafa; padding: 10px; border-radius: 8px;">
        {content}
    </div>
    """
    display(HTML(styled_content))


In [64]:
for response in responses:
    print_md_v3(response.content)
    print('*'*70)

**********************************************************************


**********************************************************************


## Stream

In [112]:
for chunk in chat.stream(messages[1]):
    print(chunk.content, end="", flush=True)

شما می‌توانید بگویید:

"Nací en Shiraz."

مثال: "Yo nací en Shiraz, una ciudad famosa por sus flores y poesía." ("من در شیراز متولد شدم، شهری که برای گل‌هایش و شعرهایش معروف است.")

In [106]:
## Add delay:
import time

for chunk in chat.stream(messages[1]):
    start = time.time()
    print(chunk.content, end="", flush=True)
    # print(time.time()-start)
    time.sleep(0.05)

شما می‌توانید بگویید:

"Nací en Shiraz."

مثال: "Yo nací en Shiraz, una ciudad famosa por sus jardines y poesía." ("من در شیراز به دنیا آمدم، شهری که به خاطر باغ‌ها و شعرهایش معروف است.")

In [110]:
## natural delay - random delay
import time
import random

for chunk in chat.stream(messages[1]):
    for char in chunk.content:
        print(char, end="", flush=True)
        time.sleep(random.uniform(0.03, 0.08))  # Vary typing speed


شما می‌توانید بگویید:

"Nací en Shiraz."

مثال: "Yo nací en Shiraz, una ciudad famosa por sus jardines y poesía." ("من در شیراز به دنیا آمدم، شهری که برای باغ‌هایش و شعرهایش معروف است.")

## asynchronnizing

In [116]:
import asyncio

async def stream_response():
    async for chunk in chat.astream(messages[1]):
        print(chunk.content, end="", flush=True)
        await asyncio.sleep(0.05)  # Slow down output

# Use `await` inside Jupyter instead of `asyncio.run()`
await stream_response()


شما می‌توانید بگویید:

"Nací en Shiraz."

مثال: "Yo nací en Shiraz, una ciudad famosa por sus flores y poesía." ("من در شیراز متولد شدم، شهری که برای گل‌هایش و شعرهایش معروف است.")